In [1]:
%run partialOrder.ipynb

..........................................................................
----------------------------------------------------------------------
Ran 74 tests in 0.470s

OK
................................................................................
----------------------------------------------------------------------
Ran 80 tests in 0.470s

OK
..........................................................................................
----------------------------------------------------------------------
Ran 90 tests in 0.686s

OK


In [2]:
from lark import Lark, Tree, Token

parser = Lark('''
                RELATION: "P".."Z"
                SET: "A".."O"
                ?logical_operator: special_operator
                     | logical_operator "and" special_operator  -> conjunction
                     | logical_operator "or"  special_operator  -> disjunction                   
                     | logical_operator "->"  special_operator  -> implication
                     | logical_operator "=="  special_operator  -> equal
                     | logical_operator "!="  special_operator  -> not_equal
                     | logical_operator ">"   special_operator  -> greater_than
                     | logical_operator "<"   special_operator  -> lower_than
                     | logical_operator ">="  special_operator  -> greater_equal
                     | logical_operator "<="  special_operator  -> lower_equal                   
                ?special_operator: properties
                     | "not" logical_operator      -> negation
                     | product "is" properties     -> is
                     | product "is not" properties -> is_not
                ?properties: product    
                     | "total_order"   -> total_order
                     | "transitive"    -> transitive
                     | "lattice"       -> lattice
                     | "reflexive"     -> reflexive
                     | "symmetric"     -> symmetric
                     | "asymmetric"    -> asymmetric
                     | "antisymmetric" -> antisymmetric
                ?product: atom
                     | product "°" atom   -> composition
                     | product "&" atom   -> intersection
                     | product "|" atom   -> union
                     | product "-" atom   -> difference
                     | product "^" atom   -> symmetric_difference
                     | product "*" atom   -> multiplication
                ?atom: "("logical_operator")"
                     | SET 
                     | RELATION
                %import common.WS
                %ignore WS
         ''', start='logical_operator')

In [3]:
def constant(data):
        try:
            return globals()[data]
        except:
            raise Exception(f"Global variable - '{data}' doesn't exist!")

def interpret(node):
    if isinstance(node, Tree):
        if node.data == 'negation':
            return not interpret(node.children[0])
        elif len(node.children) == 2:
            left, right = node.children  
            if node.data == 'equal':
                return interpret(left) == interpret(right)
            elif node.data == 'not_equal':
                return interpret(left) != interpret(right)
            elif node.data == 'greater_than':
                return interpret(left) > interpret(right)
            elif node.data == 'lower_than':
                return interpret(left) < interpret(right)
            elif node.data == 'greater_equal':
                return interpret(left) >= interpret(right)
            elif node.data == 'lower_equal':
                return interpret(left) <= interpret(right)
            elif node.data == 'conjunction':
                return interpret(left) and interpret(right)
            elif node.data == 'disjunction':
                return interpret(left) or interpret(right)
            elif node.data == 'implication':
                return (not interpret(left)) or interpret(right)
            elif node.data == 'intersection':
                return interpret(left) & interpret(right)
            elif node.data == 'union':
                return interpret(left) | interpret(right)
            elif node.data == 'difference':
                return interpret(left) - interpret(right)
            elif node.data == 'symmetric_difference':
                return interpret(left) ^ interpret(right)
            elif node.data == 'multiplication':
                return interpret(left) * interpret(right)
            elif node.data == 'composition':
                return interpret(left).composition(interpret(right))
            elif node.data == 'is' or node.data == 'is_not':
                if right.data == 'lattice':
                    return interpret(left).is_lattice() if node.data == 'is' else not interpret(left).is_lattice()
                elif right.data == 'total_order':
                    return interpret(left).is_total_order() if node.data == 'is' else not interpret(left).is_total_order()
                elif right.data == 'reflexive':
                    return interpret(left).is_reflexive() if node.data == 'is' else not interpret(left).is_reflexive()
                elif right.data == 'symmetric':
                    return interpret(left).is_symmetric() if node.data == 'is' else not interpret(left).is_symmetric()
                elif right.data == 'asymmetric':
                    return interpret(left).is_asymmetric() if node.data == 'is' else not interpret(left).is_asymmetric()
                elif right.data == 'antisymmetric':
                    return interpret(left).is_antisymmetric() if node.data == 'is' else not interpret(left).is_antisymmetric()
                elif right.data == 'transitive':
                    return interpret(left).is_transitive() if node.data == 'is' else not interpret(left).is_transitive()
            else:
                raise Exception(f"ERROR! {node.data}")
    elif isinstance(node, Token):
        node = constant(node.value)
        return node
    else:
        raise Exception(f"ERROR! unkonown instance {type(node)}")

In [4]:
R = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
S = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

tree = parser.parse("R is transitive and S is transitive -> R | S is transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpret(tree)}]")

[TREE]
implication
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  is
    union
      R
      S
    transitive
[/TREE]

[RESULT -> True]


In [5]:
R = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
S = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

tree = parser.parse("R is transitive and S is transitive and R | S is not transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpret(tree)}]")

[TREE]
conjunction
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  is_not
    union
      R
      S
    transitive
[/TREE]

[RESULT -> False]


In [6]:
R = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
S = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)

tree = parser.parse("R is transitive and S is transitive and not R | S is transitive")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpret(tree)}]")

[TREE]
conjunction
  conjunction
    is
      R
      transitive
    is
      S
      transitive
  negation
    is
      union
        R
        S
      transitive
[/TREE]

[RESULT -> False]


In [7]:
A = FiniteSet(1,2,3,4)
B = FiniteSet(3,4,5,6)
    
R = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
S = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

tree = parser.parse("S is lattice and R|S is transitive ->  S is total_order")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpret(tree)}]")

[TREE]
implication
  conjunction
    is
      S
      lattice
    is
      union
        R
        S
      transitive
  is
    S
    total_order
[/TREE]

[RESULT -> True]


In [8]:
A = FiniteSet(1,2,3,4)
B = FiniteSet(3,4,5,6)
    
R = HomogeneousRelation({(1,1),(2,2),(3,3)},{1,2,3}, True)
S = PartialOrder({(1,1),(1,2),(1,3),(2,2),(2,3),(3,3)}, {1,2,3},True)

tree = parser.parse("S != S ->  S is total_order")

print(f"[TREE]\n{tree.pretty()}[/TREE]\n")
print(f"[RESULT -> {interpret(tree)}]")

[TREE]
implication
  not_equal
    S
    S
  is
    S
    total_order
[/TREE]

[RESULT -> True]
